In [1]:
import pandas as pd
import numpy as np
import datetime
pd.options.mode.chained_assignment = None

import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore", category=RuntimeWarning)

df = pd.read_csv("Transformed/Comprehensive_SI_SSC_List.xlsx - Event_List.csv")
df = df.iloc[13:,1:5]
df = df.rename(columns={"Unnamed: 1": "Date", "Unnamed: 2": "Time", "Unnamed: 3": "DateTime", "Unnamed: 4": "Type"})
df["DateTime"] = df["DateTime"].apply(lambda x: str(x) + ":00")
df = df.replace("", np.nan, regex=True);
df = df.dropna()
df["DateTime"] = df["DateTime"].apply(lambda x : pd.to_datetime(x))
df = df.set_index("DateTime").drop(["Date", "Time"], axis=1)

In [2]:
def Trans(transformed, filename):
    now = datetime.datetime.now()
    data = pd.read_csv(transformed)
    X_train = data[['Type','dbn_nez']]
    Ind = X_train[X_train["Type"]!=0].index
    change = X_train["dbn_nez"].tolist()
    change = change[1:] + [0]
    change = change - X_train["dbn_nez"]    
    
    arrCMean = []
    arrPChange = []
    arrPoint = []  
    arrDD = []    
    
    for i in range(0, change.size):
        arrCMean.append(change.iloc[i:i+5].mean())
    X_train.insert(loc=2, column='MeanGradient', value=arrCMean)

    for i in range(0, X_train['dbn_nez'].size):    
        temp = X_train['MeanGradient'].iloc[i:i+20].abs() < 0.2
        temp.iloc[-1] = True
        temp = temp.tolist().index(True)
        temp1 = X_train['MeanGradient'].iloc[i:i+20]
        asign = np.sign(temp1)
        temp1 = ((np.roll(asign, 1) - asign) != 0).astype(int)
        temp1.iloc[0] = 0
        temp1.iloc[-1] = 1
        temp1 = temp1.tolist().index(1)
        point = min(temp, temp1) + 1
        arrPoint.append(point)  
                     
        if i+point >= X_train['dbn_nez'].size or i-(point*10)<0:
            arrDD.append(np.nan)
            arrPChange.append(np.nan) 
        else:
            arrDD.append(change.iloc[i-(point*10):i].abs().mean() / (change.iloc[i+point:i+(point*11)].abs().mean()or.0001))       
            temp  = X_train['MeanGradient'].iloc[i-(point*10):i-5]
            arrPChange.append(temp.abs().max() / (X_train['MeanGradient'].iloc[i]or.0001))   
            
    X_train.insert(loc=5, column='MaxPoint', value=arrPoint)
    X_train.insert(loc=6, column='DistRatio', value=arrDD)
    X_train.insert(loc=7, column='PreviousChange', value=arrPChange)  
    
    for i in range(0, Ind.size):
        typ = (X_train["Type"].iloc[Ind[i]])
        meanG = (X_train["MeanGradient"].iloc[Ind[i]])
        X_train["Type"].iloc[Ind[i]-2] = typ * ((X_train["MeanGradient"].iloc[Ind[i]-2] / meanG) > .6)
        X_train["Type"].iloc[Ind[i]-1] = typ * ((X_train["MeanGradient"].iloc[Ind[i]-1] / meanG) > .6)
        X_train["Type"].iloc[Ind[i]+1] = typ * ((X_train["MeanGradient"].iloc[Ind[i]+1] / meanG) > .6)
        X_train["Type"].iloc[Ind[i]+2] = typ * ((X_train["MeanGradient"].iloc[Ind[i]+2] / meanG) > .6)
    
    X_train.to_csv(filename, encoding='utf-8', index = False)
    timesince = (datetime.datetime.now()-now).total_seconds()
    print(filename, ": ", int(timesince/60), ":", int(timesince%60))

In [ ]:
#Trans("2014Transformed.csv", "ML2014.csv")
#Trans("2015Transformed.csv", "ML2015.csv")
#Trans("2016Transformed.csv", "ML2016.csv")
#Trans("2017Transformed.csv", "ML2017.csv")
#Trans("2018Transformed.csv", "ML2018.csv")
#Trans("2019Transformed.csv", "ML2019.csv")

In [ ]:
def TransOther(txt, filename):
    now = datetime.datetime.now()
    Final = pd.read_csv(filename)
    X_train = pd.read_fwf(txt)
    change = X_train["SYM/H"].tolist()
    change = change[1:] + [0]
    change = change - X_train["SYM/H"]    
    
    arrCMean = []
    arrPChange = []
    arrPoint = []  
    arrDD = []    
    
    for i in range(0, change.size):
        arrCMean.append(change.iloc[i:i+5].mean())
    Final["MeanGOther"] = arrCMean

    for i in range(0, Final["MeanGOther"].size):    
        temp = Final['MeanGOther'].iloc[i:i+20].abs() < 0.2
        temp.iloc[-1] = True
        temp = temp.tolist().index(True)
        temp1 = Final['MeanGOther'].iloc[i:i+20]
        asign = np.sign(temp1)
        temp1 = ((np.roll(asign, 1) - asign) != 0).astype(int)
        temp1.iloc[0] = 0
        temp1.iloc[-1] = 1
        temp1 = temp1.tolist().index(1)
        point = min(temp, temp1) + 1
        arrPoint.append(point)  
                     
        if i+point >= Final['MeanGOther'].size or i-(point*10)<0:
            arrDD.append(np.nan)
            arrPChange.append(np.nan) 
        else:
            arrDD.append(change.iloc[i-(point*10):i].abs().mean() / (change.iloc[i+point:i+(point*11)].abs().mean()or.0001))       
            temp  = Final['MeanGOther'].iloc[i-(point*10):i-5]
            arrPChange.append(temp.abs().max() / (Final['MeanGOther'].iloc[i]or.0001))   
            
    Final["MaxPointOther"] = arrPoint
    Final["DistRatOther"] = arrDD
    Final["PChangeOther"] = arrPChange  
    
    Final.to_csv(filename, encoding='utf-8', index = False)
    timesince = (datetime.datetime.now()-now).total_seconds()
    print(filename, ": ", int(timesince/60), ":", int(timesince%60))

In [ ]:
#TransOther("2014.txt", "ML2014.csv")
#TransOther("2015.txt", "ML2015.csv")
#TransOther("2016.txt", "ML2016.csv")
#TransOther("2017.txt", "ML2017.csv")
#TransOther("2018.txt", "ML2018.csv")
#TransOther("2019.txt", "ML2019.csv")

In [34]:
def TransOther2(txt, filename, write):
    Final = pd.read_csv(filename)
    X_train = pd.read_fwf(txt)
    Leg = pd.DataFrame(index=range(525600), columns=X_train.columns)

    for i in range(0, X_train.size):    
        Leg.iloc[i*60:(i*60)+60] = X_train.iloc[i:i+1]
    Leg = Leg.drop(columns=['YEAR', 'DAY', 'H0'])    
    Leg = pd.concat([Final, Leg], axis = 1)
    Leg.to_csv(write, encoding='utf-8', index = False)

In [37]:
TransOther2("2014.txt", "ML2014.csv", "NewML/ML2014.csv")
TransOther2("2015.txt", "ML2015.csv", "NewML/ML2015.csv")
TransOther2("2016.txt", "ML2016.csv", "NewML/ML2016.csv")
TransOther2("2017.txt", "ML2017.csv", "NewML/ML2017.csv")
TransOther2("2018.txt", "ML2018.csv", "NewML/ML2018.csv")
TransOther2("2019.txt", "ML2019.csv", "NewML/ML2019.csv")

In [31]:
def TransOther3(txt, filename, write):
    Final = pd.read_csv(filename)
    X_train = pd.read_fwf(txt)
    X_train.columns = ['Year', 'Day', 'Hour', 'Min', 'BZE', 'BZM', 'Speed', 'Press','AE', 'AL']
    X_train = X_train.drop(columns=['Year', 'Day', 'Hour', 'Min']) 
    X_train = X_train.replace([9999.99, 99999.9, 99.99, 99999] , np.nan) 
    X_train = X_train.fillna(method='ffill')
    X_train = X_train.fillna(method='bfill')
    X_train = pd.concat([Final, X_train], axis = 1)
    X_train.to_csv(write, encoding='utf-8', index = False)

In [32]:
TransOther3("2014.txt", "ML2014.csv", "NewML/ML2014.csv")
TransOther3("2015.txt", "ML2015.csv", "NewML/ML2015.csv")
TransOther3("2016.txt", "ML2016.csv", "NewML/ML2016.csv")
TransOther3("2017.txt", "ML2017.csv", "NewML/ML2017.csv")
TransOther3("2018.txt", "ML2018.csv", "NewML/ML2018.csv")
TransOther3("2019.txt", "ML2019.csv", "NewML/ML2019.csv")

In [1]:
# X_train = pd.read_csv("NewML/ML2019.csv")
# X_train